# Query Examples
This notebook contains a few examples of search performed on the `books` index previously created from the files [local_embedding_example](./local_embedding_example.ipynb) or [pipeline-inference-example](./pipeline-inference-example.ipynb). 

In [ ]:
!pip install -r requirements.txt

In [2]:
from elasticsearch import Elasticsearch
from getpass import getpass
import pprint

pp = pprint.PrettyPrinter(indent=2)

Connect to Elasticsearch with an Elastic Deployemnt Endpoint and Elastic Api Key

This notebook connects to the `books-pipeline` index, as the ingestion-based scripts index all of the books by default. If you would like to use the `books-local` index instead, replace the value of `INDEX_NAME` to `books-local` below.

In [ ]:
cloud_endpoint = getpass("Elastic deployment Cloud Endpoint: ")
cloud_api_key = getpass("Elastic deployment API Key: ")

INDEX_NAME = 'books-pipeline'
MODEL_ID = 'sentence-transformers__msmarco-minilm-l-12-v3'

es = Elasticsearch(
    hosts=cloud_endpoint,
    api_key=cloud_api_key,
)

query_string = "Dinosaurs are still alive"

#### The queries will return and display using the `print_results` function defined below which outlines the book title, description, and search return score.

In [6]:
def print_results(search_result):
    print(f"Total hits: {search_result['hits']['total']['value']}")
    for hit in search_result["hits"]["hits"]:
        print(f"Book: {hit['_source']['book_title']}")
        print(f"Author: {hit['_source']['author_name']}")
        print(f"Description: {hit['_source']['book_description']}")
        print(f"Score: {hit['_score']}\n")


### 1. Vector Search Example

In [ ]:
vector_search_result = es.search(
        index = INDEX_NAME,
        knn={
            "field": "description_embedding",
            "k": 10,
            "num_candidates": 50,
            "query_vector_builder": {
                "text_embedding": {"model_id": MODEL_ID, "model_text": query_string}
            },
        },
    )

print_results(vector_search_result)

### 2. Full-Text Search Example

In [ ]:
bm25_result = es.search(
        index=INDEX_NAME, 
        body={
            "query": {
                "match": {
                    "book_description": query_string
                }
            },
            "size": 10
        },
        
    )

print_results(bm25_result)

### 3. Hybrid Search Example

In [ ]:
hybrid_search_result = es.search(
    index=INDEX_NAME,
    body={
        "query": {
            "bool": {
                "must": [
                    {"match": 
                        {
                            "book_description": query_string,
                        }
                    },
                ]
            }
        },
        "knn": {
            "field": "description_embedding",
            "k": 10,
            "query_vector_builder": {
                "text_embedding": {"model_id": MODEL_ID, "model_text": query_string}
            },
        },
        "rank": {
            "rrf": {}
        },
        "size": 10
    },
)

print_results(hybrid_search_result)